# EEG Statistics

Using some of the data you collected, we are going to go through a couple of different ways to discern whether the difference in signal between your conditions is statistically significant.

#### Setting up Python
Before starting to analyse our own EEG data, we need to make sure we have our virtual environment we created during the `MNE-tutorial`.

1. Press `Select Kernel`, then `Python Environments...` and then choose any Python kernel. 
2. Run the code chunk below
3. Change the kernel used to run the code in this notebook. Press where it says `Python X.XX.XX` in the top right corner, then `Select Another Kernel`, then `Jupyter kernel...` and then select `env`. If `env` does not show up, press the little refresh symbol! 

In [ ]:
!bash ../env_to_ipynb_kernel.sh

#### Import packages

In [ ]:
import mne
import numpy as np
import pandas as pd
import os


## Epochs
We need epochs for the statistical tests. We will be using the function from earlier today to preprocess the data!

In [ ]:
def get_triggers(logfile, events):
    def map_categories(column, category_map):
        return column.map(category_map)

    def create_event_column(logfile):
        return logfile['lexical_category'] + logfile['prime_category'] + logfile['target_category']

    def create_event_rt_column(logfile):
        event_rt = logfile['correct'].replace({1: 100, 0: 105}) + logfile['event']
        return event_rt

    def separate_time_columns(logfile):
        return logfile['time_target'] + logfile['response_time_keyboard_response']

    def create_temp_events(logfile):
        temp = []
        for ev, resp in zip(logfile['event'], logfile['event_rt']):
            temp.extend([ev + 5, ev, resp])
        return temp

    #def exp_trigger_list(logfile):
    #    trigger_list = []
    #    for p, t, r in zip(logfile['time_prime_trigger'][12:], logfile['time_target_trigger'][12:], logfile['time_resp_trigger'][12:]):
    #        trigger_list.extend([p, t, r])
    #    return trigger_list

    #def prac_trigger_list(logfile):
    #    trigger_list = []
    #    for p, t, r in zip(logfile['time_prac_prime_trigger'][:12], logfile['time_prac_target_trigger'][:12], logfile['time_prac_resp_trigger'][:12]):
    #        trigger_list.extend([p, t, r])
    #    return trigger_list

    # Mapping of categorical values
    category_maps = {
        'lexical_category': {'word': 0, 'nonword': 2},
        'prime_category': {'congruent': 0, 'incongruent': 1},
        'target_category': {'practice': 90, 'compound_first': 50, 'basic': 20, 'filler': 10,
                            'abstract': 80, 'subordinate': 30, 'nontransparent_last': 60,
                            'nontransparent_first': 70, 'compound_last': 40}
    }

    # apply category mapping
    for column, category_map in category_maps.items():
        logfile[column] = map_categories(logfile[column], category_map)

    # create new columns
    logfile['event'] = create_event_column(logfile)
    logfile['event_rt'] = create_event_rt_column(logfile)
    logfile['event_rt_time'] = separate_time_columns(logfile)

    # create temporary event and trigger lists for practise and experimental trials
    temp = create_temp_events(logfile)
    #exp_trigs = exp_trigger_list(logfile)
    #prac_trigs = prac_trigger_list(logfile)


    # remove consecutive duplicates from events
    temp_dif = np.append(1, np.diff(events[:, 0]))
    tempi = np.where(temp_dif == 0)[0] - 1
    new_events = np.delete(events, tempi, 0)

    # insert new events into the new events array
    new_events[5:14*3-1, 2] = temp[:36]

    # for some reason the lengths do not always match up?? a bit concerning 
    try:
        new_events[14*3:, 2] = temp[36:]
    except:
        new_events[14*3:-1, 2] = temp[36:]

    return new_events



# function fo preprocessing one subject
def preproc_subject(raw, logfile, event_id, bad_channels = ["Fp1", "Fp2"], reject = {"eeg":150e-6}, l_freq = 1, h_freq = 40, tmin = -0.2, tmax = 1, baseline = (-0.2, None)):
    """
    Preprocesses one subject's data.

    Parameters
    ----------
    raw : mne.io.Raw
        Raw data.
    """
    # set montage
    montage = mne.channels.make_standard_montage("standard_1020")
    raw.set_montage(montage)
    
    # remove bad channels
    raw.info["bads"] = bad_channels
    raw.interpolate_bads(reset_bads = True)
    
    # bandpass filter
    raw.filter(l_freq, h_freq)
    
    # epoching
    events, _ = mne.events_from_annotations(raw, verbose=False)
    events = get_triggers(logfile, events)

    updated_event_id = get_correct_event_ids(events, event_id)

    epochs = mne.Epochs(raw, events, updated_event_id, tmin, tmax, baseline = baseline, reject = reject, preload = True)
    
    # artifact rejection
    epochs.drop_bad()
    
    return epochs

# function for getting only the event_ids found in the session
def get_correct_event_ids(events, event_id):
    # modify event id so we only have triggers from our data so we do not get an error when we are trying to run 
    triggers_unique = np.unique(events[:, 2])

    # Create a new dictionary for the selected triggers
    updated_event_ids = {}

    # Iterate through the event_id dictionary
    for key, value in event_id.items():
        if value in triggers_unique:
            updated_event_ids[key] = value

    return updated_event_ids


In [ ]:
# Load raw data 
# reading the file & loading in the data
data_folder = '/work/811462/'
group_name = "group1"

# path to the data - MAKE SURE TO CHANGE PATH ACCORDING TO YOUR GROUP NUMBER!
data_path = os.path.join(data_folder, f"{group_name}.vhdr")

raw = mne.io.read_raw_brainvision(data_path)
raw.load_data()

# set standard montage (let MNE know the layout of the electrodes on the cap)
montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage, verbose=False)


In [ ]:
# reading in the csv file with experiment information
logfile = pd.read_csv(os.path.join(data_folder, f"subject-{group_name[-1]}.csv"))
logfile.head()

In [ ]:
# the big event array hurray
event_id = {'filler/congruent/word/target': 10, # filler word with congruent prime
            'filler/congruent/nonword/target': 12, # "filler" non-word with congruent prime
            'filler/congruent/word/prime': 15, # congruent prime filler word
            'filler/congruent/nonword/prime': 17, # "congruent" prime before "filler" non-word
            'filler/congruent/word/correct': 110, # filler word with congruent prime - correct answer
            'filler/congruent/word/incorrect': 115, # filler word with congruent prime - incorrect answer
            'filler/congruent/nonword/correct': 112, # "filler" non-word with "congruent" prime - correct answer
            'filler/congruent/nonword/incorrect': 117, # "filler" non-word with "congruent" prime - incorrect answer
            'filler/incongruent/word/target': 11, # filler word with incongruent prime
            'filler/incongruent/nonword/target': 13, # "filler" non-word with "incongruent" prime
            'filler/incongruent/word/prime': 16, # incongruent prime before filler word
            'filler/incongruent/nonword/prime': 18, # "incongruent" prime before "filler" non-word
            'filler/incongruent/word/correct': 111, # filler word with incongruent prime - correct answer
            'filler/incongruent/word/incorrect': 116, # filler word with incongruent prime - incorrect answer
            'filler/incongruent/nonword/correct': 113, # "filler" non-word with "incongruent" prime - correct answer
            'filler/incongruent/nonword/incorrect': 118, # "filler" non-word with "incongruent" prime - incorrect answer
            'basic/congruent/word/target': 20, # basic-category word with congruent prime
            'basic/congruent/nonword/target': 22, # basic-category nonword with congruent prime
            'basic/congruent/word/prime': 25, # basic-category word with congruent prime
            'basic/congruent/nonword/prime': 27, # basic-category nonword with congruent prime
            'basic/congruent/word/correct': 120, # basic-category word with congruent prime - correct answer
            'basic/congruent/word/incorrect': 125, # basic-category word with congruent prime - incorrect answer
            'basic/congruent/nonword/correct': 122, # basic-category non-word with congruent prime - correct answer
            'basic/congruent/nonword/incorrect': 127, # basic-category non-word with congruent prime - incorrect answer
            'basic/incongruent/word/target': 21, # basic-category word with incongruent prime
            'basic/incongruent/nonword/target': 23, # basic-category nonword with incongruent prime
            'basic/incongruent/word/prime': 26, # basic-category word with incongruent prime
            'basic/incongruent/nonword/prime': 28, # basic-category nonword with incongruent prime
            'basic/incongruent/word/correct': 121, # basic-category word with incongruent prime - correct answer
            'basic/incongruent/word/incorrect': 126, # basic-category word with incongruent prime - incorrect answer
            'basic/incongruent/nonword/correct': 123, # basic-category non-word with incongruent prime - correct answer
            'basic/incongruent/nonword/incorrect': 128, # basic-category non-word with incongruent prime - incorrect answer
            'subordinate/congruent/word/target': 30, # subordinate word with congruent prime
            'subordinate/congruent/nonword/target': 32, # subordinate nonword with congruent prime
            'subordinate/congruent/word/prime': 35, # subordinate word with congruent prime
            'subordinate/congruent/nonword/prime': 37, # subordinate nonword with congruent prime
            'subordinate/congruent/word/correct': 130, # subordinate word with congruent prime - correct answer
            'subordinate/congruent/word/incorrect': 135, # subordinate word with congruent prime - incorrect answer
            'subordinate/congruent/nonword/correct': 132, # subordinate non-word with congruent prime - correct answer
            'subordinate/congruent/nonword/incorrect': 137, # subordinate non-word with congruent prime - incorrect answer
            'subordinate/incongruent/word/target': 31, # subordinate word with incongruent prime
            'subordinate/incongruent/nonword/target': 33, # subordinate nonword with incongruent prime
            'subordinate/incongruent/word/prime': 36, # subordinate word with incongruent prime
            'subordinate/incongruent/nonword/prime': 38, # subordinate nonword with incongruent prime
            'subordinate/incongruent/word/correct': 131, # subordinate word with incongruent prime - correct answer
            'subordinate/incongruent/word/incorrect': 136, # subordinate word with incongruent prime - incorrect answer
            'subordinate/incongruent/nonword/correct': 133, # subordinate non-word with incongruent prime - correct answer
            'subordinate/incongruent/nonword/incorrect': 138, # subordinate non-word with incongruent prime - incorrect answer
            'compoundlast/congruent/word/target': 40, # compound-last word with congruent prime
            'compoundlast/congruent/nonword/target': 42, # compound-last nonword with congruent prime
            'compoundlast/congruent/word/prime': 45, # compound-last word with congruent prime
            'compoundlast/congruent/nonword/prime': 47, # compound-last nonword with congruent prime
            'compoundlast/congruent/word/correct': 140, # compound-last word with congruent prime - correct answer
            'compoundlast/congruent/word/incorrect': 145, # compound-last word with congruent prime - incorrect answer
            'compoundlast/congruent/nonword/correct': 142, # compound-last non-word with congruent prime - correct answer
            'compoundlast/congruent/nonword/incorrect': 147, # compound-last non-word with congruent prime - incorrect answer
            'compoundlast/incongruent/word/target': 41, # compound-last word with incongruent prime
            'compoundlast/incongruent/nonword/target': 43, # compound-last nonword with incongruent prime
            'compoundlast/incongruent/word/prime': 46, # compound-last word with incongruent prime
            'compoundlast/incongruent/nonword/prime': 48, # compound-last nonword with incongruent prime
            'compoundlast/incongruent/word/correct': 141, # compound-last word with incongruent prime - correct answer
            'compoundlast/incongruent/word/incorrect': 146, # compound-last word with incongruent prime - incorrect answer
            'compoundlast/incongruent/nonword/correct': 143, # compound-last non-word with incongruent prime - correct answer
            'compoundlast/incongruent/nonword/incorrect': 148, # compound-last non-word with incongruent prime - incorrect answer
            'compoundfirst/congruent/word/target': 50, # compound-first word with congruent prime
            'compoundfirst/congruent/nonword/target': 52, # compound-first nonword with congruent prime
            'compoundfirst/congruent/word/prime': 55, # compound-first word with congruent prime
            'compoundfirst/congruent/nonword/prime': 57, # compound-first nonword with congruent prime
            'compoundfirst/congruent/word/correct': 150, # compound-first word with congruent prime - correct answer
            'compoundfirst/congruent/word/incorrect': 155, # compound-first word with congruent prime - incorrect answer
            'compoundfirst/congruent/nonword/correct': 152, # compound-first non-word with congruent prime - correct answer
            'compoundfirst/congruent/nonword/incorrect': 157, # compound-first non-word with congruent prime - incorrect answer
            'compoundfirst/incongruent/word/target': 51, # compound-first word with incongruent prime
            'compoundfirst/incongruent/nonword/target': 53, # compound-first nonword with incongruent prime
            'compoundfirst/incongruent/word/prime': 56, # compound-first word with incongruent prime
            'compoundfirst/incongruent/nonword/prime': 58, # compound-first nonword with incongruent prime
            'compoundfirst/incongruent/word/correct': 151, # compound-first word with incongruent prime - correct answer
            'compoundfirst/incongruent/word/incorrect': 156, # compound-first word with incongruent prime - incorrect answer
            'compoundfirst/incongruent/nonword/correct': 153, # compound-first non-word with incongruent prime - correct answer
            'compoundfirst/incongruent/nonword/incorrect': 158, # compound-first non-word with incongruent prime - incorrect answer
            'nontransparentlast/congruent/word/target': 60, # nontransparent compound-last word with congruent prime
            'nontransparentlast/congruent/nonword/target': 62, # "nontransparent compound-last" non-word with "congruent" prime
            'nontransparentlast/congruent/word/prime': 65, # congruent prime before nontransparent compound-last word
            'nontransparentlast/congruent/nonword/prime': 67, # "congruent" prime before "nontransparent compound-last" non-word
            'nontransparentlast/congruent/word/correct': 160, # nontransparent compound-last word with congruent prime - correct answer
            'nontransparentlast/congruent/word/incorrect': 165, # nontransparent compound-last word with congruent prime - incorrect answer
            'nontransparentlast/congruent/nonword/correct': 162, # "nontransparent compound-last" non-word with "congruent" prime - correct answer
            'nontransparentlast/congruent/nonword/incorrect': 167, # "nontransparent compound-last" non-word with "congruent" prime - incorrect answer
            'nontransparentlast/incongruent/word/target': 61, # nontransparent compound-last word with incongruent prime
            'nontransparentlast/incongruent/nonword/target': 63, # nontransparent compound-last non-word with "incongruent" prime
            'nontransparentlast/incongruent/word/prime': 66, # incongruent prime before nontransparent compound-last word
            'nontransparentlast/incongruent/nonword/prime': 68, # "incongruent" prime before "nontransparent compound-last" non-word
            'nontransparentlast/incongruent/word/correct': 161, # nontransparent compound-last word with incongruent prime - correct answer
            'nontransparentlast/incongruent/word/incorrect': 166, # nontransparent compound-last word with incongruent prime - incorrect answer
            'nontransparentlast/incongruent/nonword/correct': 163, # "nontransparent compound-last" non-word with "incongruent" prime - correct answer
            'nontransparentlast/incongruent/nonword/incorrect': 168, # "nontransparent compound-last" non-word with "incongruent" prime - incorrect answer
            'nontransparentfirst/congruent/word/target': 70, # nontransparent compound-first word with congruent prime
            'nontransparentfirst/congruent/nonword/target': 72, # "nontransparent compound-first" non-word with "congruent" prime
            'nontransparentfirst/congruent/word/prime': 75, # congruent prime before nontransparent compound-first word
            'nontransparentfirst/congruent/nonword/prime': 77, # congruent prime before "nontransparent compound-first" non-word
            'nontransparentfirst/congruent/word/correct': 170, # nontransparent compound-first word with congruent prime - correct answer
            'nontransparentfirst/congruent/word/incorrect': 175, # nontransparent compound-first word with congruent prime - incorrect answer
            'nontransparentfirst/congruent/nonword/correct': 172, # "nontransparent compound-first" non-word with "congruent" prime - correct answer
            'nontransparentfirst/congruent/nonword/incorrect': 177, # "nontransparent compound-first" non-word with "congruent" prime - incorrect answer
            'nontransparentfirst/incongruent/word/target': 71, # nontransparent compound-first word with incongruent prime
            'nontransparentfirst/incongruent/nonword/target': 73, # nontransparent compound-first non-word with "incongruent" prime
            'nontransparentfirst/incongruent/word/prime': 76, # incongruent prime before nontransparent compound-first word
            'nontransparentfirst/incongruent/nonword/prime': 78, # "incongruent" prime before "nontransparent compound-first" non-word
            'nontransparentfirst/incongruent/word/correct': 171, # nontransparent compound-first word with incongruent prime - correct answer
            'nontransparentfirst/incongruent/word/incorrect': 176, # nontransparent compound-first word with incongruent prime - incorrect answer
            'nontransparentfirst/incongruent/nonword/correct': 173, # "nontransparent compound-first" non-word with "incongruent" prime - correct answer
            'nontransparentfirst/incongruent/nonword/incorrect': 178, # "nontransparent compound-first" non-word with "incongruent" prime - incorrect answer
            'abstract/congruent/word/target': 80, # abstract word with congruent prime
            'abstract/congruent/nonword/target': 82, # abstract non-word with "congruent" prime
            'abstract/congruent/word/prime': 85, # congruent prime before abstract target
            'abstract/congruent/nonword/prime': 87, # "congruent prime" before "abstract" non-word
            'abstract/congruent/word/correct': 180, # abstract word with congruent prime - correct answer
            'abstract/congruent/word/incorrect': 185, # abstract word with congruent prime - incorrect answer
            'abstract/congruent/nonword/correct': 182, # abstract non-word with congruent prime - correct answer
            'abstract/congruent/nonword/incorrect': 187, # abstract non-word with congruent prime - incorrect answer
            'abstract/incongruent/word/target': 81, # abstract word with incongruent prime
            'abstract/incongruent/nonword/target': 83, # abstract non-word with "incongruent" prime
            'abstract/incongruent/word/prime': 86, # incongruent prime abstract word
            'abstract/incongruent/nonword/prime': 88, # "incongruent" prime before "abstract" non-word
            'abstract/incongruent/word/correct': 181, # abstract word with incongruent prime - correct answer
            'abstract/incongruent/word/incorrect': 186, # abstract word with incongruent prime - incorrect answer
            'abstract/incongruent/nonword/correct': 183, # abstract non-word with "incongruent" prime - correct answer
            'abstract/incongruent/nonword/incorrect': 188, # abstract non-word with "incongruent" prime - incorrect answer
            'practice/congruent/word/target': 90, # practice word with congruent prime
            'practice/congruent/nonword/target': 92, # practice non-word with "congruent" prime
            'practice/congruent/word/prime': 95, # "congruent" prime before practice word
            'practice/congruent/nonword/prime': 97, # "congruent" prime before practice non-word
            'practice/congruent/word/correct': 190, # practice word with congruent prime - correct answer
            'practice/congruent/word/incorrect': 195, # practice word with congruent prime - incorrect answer
            'practice/congruent/nonword/correct': 192, # practice non-word with "congruent" prime - correct answer
            'practice/congruent/nonword/incorrect': 197, # practice non-word with "congruent" prime - incorrect answer
            'practice/incongruent/word/target': 91, # practice word with incongruent prime
            'practice/incongruent/nonword/target': 93, # practice non-word with "incongruent" prime
            'practice/incongruent/word/prime': 96, # incongruent prime before practice word
            'practice/incongruent/nonword/prime': 98, # "incongruent" prime before practice non-word
            'practice/incongruent/word/correct': 191, # practice word with incongruent prime - correct answer
            'practice/incongruent/word/incorrect': 196, # practice word with incongruent prime - incorrect answer
            'practice/incongruent/nonword/correct': 193, # practice non-word with "incongruent" prime - correct answer
            'practice/incongruent/nonword/incorrect': 198 # practice non-word with "incongruent" prime - incorrect answer
           }

In [ ]:
# Lets preprocess the data
epochs = preproc_subject(
    raw, 
    logfile,
    event_id, 
    bad_channels = ["Fp1", "Fp2"], 
    reject = {"eeg":150e-6}, 
    l_freq = 1, 
    h_freq = 40, 
    tmin = -0.2, 
    tmax = 1, 
    baseline = (-0.2, None)
    )

Okay, now it is time for you to make some decisions! At this point you need to choose a contrast:
1. 'word/target' & 'nonword/target'

2. 'word/congruent' & 'word/incongruent'

3. 'word/prime' & 'word/target'


In [ ]:
# Extract the contrast you are interested in using the following code as an example

epochs_condition1 = epochs["word/target"]
epochs_condition2 = epochs["nonword/target"]

## Windowed mean
Now we have our two conditions: trials with words vs images. One of the simplest way in which we can determine whether the signal in our two conditions are statistically significant is by:

1) Segmenting our data using only certain channels in a specific time window. Keep in mind that which time window and channels should be established a priori, for instance according to the literature. 
2) Taking the mean of that window across channels and and samples.
3) Running statistical tests on the windowed means from the two conditions.

In an experiment with multiple participants we would also average over trials from individual participants, in order to only have one data point per participant (and thereby avoid multiple comparisons). However, since we have one participant, we can keep one dimension of the individual data, i.e. the trials.

### T-test
We can now do a t-test on the trials from the two conditions, to establish whether the means of the two groups are statistically significant.

We can use the get_data() function to get the numerical values of the signal (in microvolts) for the t-test. tmin and tmax are used to define the size of the window, and the picks are the channels that we expect to see an effect in.

In [ ]:
# INSERT CHANNEL NAMES YOU ARE INTERESTED IN BELOW
picks = ["xxx", "xxx", "xxx"]


# DETERMINE THE TIME FRAME YOU WANT TO LOOK AT (remember to write it in seconds )
tmin = # insert number here
tmax = # insert number here


In [ ]:
# Now we can extract the data using the following logic
data_condition1 = epochs_condition1.get_data(picks = picks, tmin = tmin, tmax = tmax)
data_condition2 = epochs_condition2.get_data(picks = picks, tmin = tmin, tmax = tmax)

Investigating the resulting data; how many dimensions does the data have? What do you think they represent (i.e. which dimension is channels, trials, etc.)?

In [ ]:
print(data_condition1.shape)
print(data_condition2.shape)

Now we can average over the data so we only have the trials dimension.

In [ ]:
data_condition1_mean = np.mean(data_condition1, axis=2) # averaging over the third dimension of the data
print(data_condition1_mean.shape)

data_condition1_mean = np.mean(data_condition1_mean, axis=1) # averaging over the second dimension of the data
print(data_condition1_mean.shape)

## Now do the same for the second condition!

# INSERT CODE HERE

### Running the t-test

In [ ]:
# installing additional packages
!pip install scipy
from scipy import stats as st
import statistics as stats

In [ ]:
# running the t-test
st.ttest_ind(a=data_condition1_mean, b=data_condition2_mean)